### Librerías

In [1]:
# Librerías
import pandas as pd
import requests
import datetime

# !pip install geopandas
import geopandas as gpd
from difflib import SequenceMatcher # similitud de cadenas
import re                           # expresiones regulares
import chardet                      # codificación del fichero
from shapely.wkt import loads       # genera geometrías

C:\Users\34686\Downloads\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\34686\Downloads\Anaconda\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\34686\Downloads\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\34686\Downloads\Anaconda\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-1-f60070fc1914>:7: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set t

Links:
* https://valencia.opendatasoft.com/explore/dataset/fgv-estacions-estaciones/export/ 

* https://valencia.opendatasoft.com/explore/dataset/estacions-rodalia-valencia-val/export/

* https://valencia.opendatasoft.com/explore/dataset/emt/table/

* https://valencia.opendatasoft.com/explore/dataset/transporte-barrios/table/

### Lectura de datos

In [2]:
# Nombre de los ficheros
metro = "Datos transporte/fgv-estacions-estaciones.geojson"
emt = "Datos transporte/transporte-barrios.geojson"
renfe = "Datos transporte/estacions-rodalia-valencia-val.geojson"
emt2 = "Datos transporte/emt.geojson"
emt3 = "Datos transporte/emt_concatenado_geometry.csv"

# Lectura de datos
gdf_met = gpd.read_file(metro)
gdf_emt = gpd.read_file(emt, encoding="utf-8")
gdf_ren = gpd.read_file(renfe)
gdf_emt2 = gpd.read_file(emt2)

### Primera visualización

In [3]:
# Metrovalencia
gdf_met.head()

,gid,codigo,nombre,tipo,linea,proximas_llegadas,proximes_llegadas,geo_point_2d,geometry
0,237118,110,Mas del Rosari,1,4,https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.43582499030541805, 'lat': 39.524959...",POINT (-0.43582 39.52496)
1,237200,183,La Cova,1,9,https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.48444443970429724, 'lat': 39.498889...",POINT (-0.48444 39.49889)
2,237215,121,Aeroport,1,"3,5",https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.47491943881419474, 'lat': 39.492366...",POINT (-0.47492 39.49237)
3,237216,8,Almàssera,1,3,https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.3542666730314796, 'lat': 39.5122566...",POINT (-0.35427 39.51226)
4,237142,15,Colón,1,"3,5,7,9",https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.3709277814305924, 'lat': 39.4701462...",POINT (-0.37093 39.47015)


In [4]:
# Emt
gdf_emt.head()

,geo_point_2d,geo_shape,codbarrio,nombre,coddistbar,coddistrit,codbar,transporte,stop_id,stop_name,stop_desc,x,y,zone_id,geometry
0,"{'lon': -0.38580384886348595, 'lat': 39.527643...",None,1,BENIFARAIG,171,17,171,emt,1137.0,Ferrer i Bign� - Aiacor,C FERRER Y BIGNE 21 - BENIFARAIG,724691.91,4378596.09,None,POINT (-0.38580 39.52764)
1,"{'lon': -0.3993369274208722, 'lat': 39.4910315...",None,1,BENICALAP,161,16,161,metrovlc,54.0,Beniferri,NULL,723646.02,4374498.31,A,POINT (-0.39934 39.49103)
2,"{'lon': -0.3872583065788684, 'lat': 39.4895628...",None,1,BENICALAP,161,16,161,metrovlc,96.0,Tr�nsits,NULL,724689.60,4374365.36,A,POINT (-0.38726 39.48956)
3,"{'lon': -0.39681665593963605, 'lat': 39.494434...",None,1,BENICALAP,161,16,161,metrovlc,99.0,Florista,NULL,723851.86,4374882.28,A,POINT (-0.39682 39.49443)
4,"{'lon': -0.3927782255244965, 'lat': 39.4960788...",None,1,BENICALAP,161,16,161,emt,1067.0,Equador - Pintor Matarana,AV ECUADOR EL 93 (DAVANT) - VAL�NCIA,724193.89,4375074.89,None,POINT (-0.39278 39.49608)


In [5]:
# Rodalies
gdf_ren.head()

,geo_point_2d,geo_shape,codi,descripci,latitud,longitud,direcci,geometry
0,"{'lon': -0.320388, 'lat': 39.5441373}",None,65005,ALBUIXECH,39.5441373,-0.320388,AVENIDA VIRGEN ALBUIXECH,POINT (-0.32039 39.54414)
1,"{'lon': -0.0775039, 'lat': 39.9543523}",None,65209,ALMASSORA,39.9543523,-0.0775039,CARRER ESTACIO FERROCARRIL,POINT (-0.07750 39.95435)
2,"{'lon': -0.2156704, 'lat': 39.7522591}",None,65202,ALMENARA,39.7522591,-0.2156704,CALLE ESTACION FERROCARRIL,POINT (-0.21567 39.75226)
3,"{'lon': -0.7875467, 'lat': 39.4273336}",None,66206,BUÑOL,39.4273336,-0.7875467,CALLE ESTACION FERROCARRIL,POINT (-0.78755 39.42733)
4,"{'lon': -0.4547076, 'lat': 39.1182487}",None,64103,CARCAIXENT,39.1182487,-0.4547076,GLORIETA ESTACION FERROCARRIL,POINT (-0.45471 39.11825)


### Modificación de atributos
Para poder mergearlos posteriormente y tener solo uno con los siguientes atributos:
* id
* nombre
* longitud
* latitud
* geometry
* linea
* tipo
* municipio

In [6]:
# Metrovalencia

def coordenadas_long(x):
    return x['lon']
def coordenadas_lat(x):
    return x['lat']


gdf_met['longitud'] = list(map(coordenadas_long, gdf_met.geo_point_2d))
gdf_met['latitud'] = list(map(coordenadas_lat, gdf_met.geo_point_2d))


columnas_eliminar = ['codigo', 'tipo', 'proximas_llegadas', 'proximes_llegadas', 'geo_point_2d']
gdf_met.drop(columnas_eliminar, axis=1, inplace=True)

gdf_met.rename(columns={'gid': 'id'}, inplace=True)

gdf_met['tipo'] = 'metrovalencia'
gdf_met['municipio'] = None

gdf_met.head()

,id,nombre,linea,geometry,longitud,latitud,tipo,municipio
0,237118,Mas del Rosari,4,POINT (-0.43582 39.52496),-0.435825,39.524960,metrovalencia,None
1,237200,La Cova,9,POINT (-0.48444 39.49889),-0.484444,39.498890,metrovalencia,None
2,237215,Aeroport,"3,5",POINT (-0.47492 39.49237),-0.474919,39.492367,metrovalencia,None
3,237216,Almàssera,3,POINT (-0.35427 39.51226),-0.354267,39.512257,metrovalencia,None
4,237142,Colón,"3,5,7,9",POINT (-0.37093 39.47015),-0.370928,39.470146,metrovalencia,None


In [7]:
# Renfe

   
no = {"L", "LA", "EL", "DE", "DEL", "DELS", "I"}

def palabra_con_apostrof_guionet(palabra):
    list_pal = palabra.split("-")
    res = []
    for i, pal in enumerate(list_pal):
        apostrof = pal.find("'")
        if apostrof != -1:
            res.append(pal[0].upper()+pal[1]+pal[2:].capitalize())
        elif pal in no:
            res.append(pal.lower())
        else:
            res.append(pal.capitalize())
    palabra = "-".join(res)
    return palabra

def palabra_con_guionet(palabra):
    list_pal = palabra.split("-")
    list_pal = [pal.capitalize() for pal in list_pal]
    palabra = "-".join(list_pal)
    return palabra
   
    
def capitalizar(palabra):
    if palabra is None:
        return None
    
    list_pal = palabra.split()
    res = []
    for i, pal in enumerate(list_pal):
        apostrof = pal.find("'")
        guionet = pal.find("-")
                       
        if i==0 and pal in no:
            res.append(pal.capitalize())
        elif guionet!=-1 and apostrof!=-1:            
            res.append(palabra_con_apostrof_guionet(pal))
        elif apostrof!=-1:
            res.append(palabra[apostrof-1].upper()+palabra[apostrof]+palabra[apostrof+1:].capitalize())
        elif guionet!=-1:
            res.append(palabra_con_guionet(pal))
        elif pal in no:
            res.append(pal.lower())
        else:
            res.append(pal.capitalize())
    res = " ".join(res)
    return res
    

# Eliminar columnas
columnas_eliminar = ['geo_point_2d', 'geo_shape', 'direcci']
gdf_ren.drop(columnas_eliminar, axis=1, inplace=True)

# Renombrar columnas
gdf_ren.rename(columns={'codi': 'id'}, inplace=True)
gdf_ren.rename(columns={'descripci': 'municipio'}, inplace=True)

# Añadir columnas
gdf_ren['tipo'] = 'renfe'
gdf_ren['linea'] = None

# Modificar columnas
gdf_ren['municipio'] = list(map(capitalizar, gdf_ren.municipio))
gdf_ren['nombre'] = gdf_ren.municipio

gdf_ren.head()

,id,municipio,latitud,longitud,geometry,tipo,linea,nombre
0,65005,Albuixech,39.5441373,-0.320388,POINT (-0.32039 39.54414),renfe,None,Albuixech
1,65209,Almassora,39.9543523,-0.0775039,POINT (-0.07750 39.95435),renfe,None,Almassora
2,65202,Almenara,39.7522591,-0.2156704,POINT (-0.21567 39.75226),renfe,None,Almenara
3,66206,Buñol,39.4273336,-0.7875467,POINT (-0.78755 39.42733),renfe,None,Buñol
4,64103,Carcaixent,39.1182487,-0.4547076,POINT (-0.45471 39.11825),renfe,None,Carcaixent


In [8]:
# Emt

def coordenadas_long(x):
    if x is not None:
        return x['lon']
def coordenadas_lat(x):
    if x is not None:
        return x['lat']

# Añadir columnas
gdf_emt['longitud'] = list(map(coordenadas_long, gdf_emt.geo_point_2d))
gdf_emt['latitud'] = list(map(coordenadas_lat, gdf_emt.geo_point_2d))
gdf_emt['linea'] = None

# Quitar filas
filas_eliminar = gdf_emt[gdf_emt['transporte']!='emt'].index
gdf_emt.drop(filas_eliminar, axis=0, inplace=True)

# Quitar columnas
columnas_eliminar = ['geo_point_2d', 'geo_shape', 'codbarrio', 'coddistbar', 'coddistrit', 'codbar', 'stop_desc', 'x', 'y', 'zone_id']
gdf_emt.drop(columnas_eliminar, axis=1, inplace=True)

# Renombrar columnas
gdf_emt.rename(columns={'stop_id': 'id'}, inplace=True)
gdf_emt.rename(columns={'stop_name': 'direccion'}, inplace=True)
gdf_emt.rename(columns={'transporte': 'tipo'}, inplace=True)
gdf_emt.rename(columns={'nombre': 'municipio'}, inplace=True)
gdf_emt.rename(columns={'direccion': 'nombre'}, inplace=True)

# Modificar columna
gdf_emt['municipio'] = list(map(capitalizar, gdf_emt.municipio))

gdf_emt.head()

,municipio,tipo,id,nombre,geometry,longitud,latitud,linea
0,Benifaraig,emt,1137.0,Ferrer i Bign� - Aiacor,POINT (-0.38580 39.52764),-0.385804,39.527644,None
4,Benicalap,emt,1067.0,Equador - Pintor Matarana,POINT (-0.39278 39.49608),-0.392778,39.496079,None
5,Benicalap,emt,1183.0,General Avil�s (par) - Dr. Nicasi Benlloch,POINT (-0.39146 39.48804),-0.391461,39.488037,None
6,Benicalap,emt,1319.0,Corts Valencianes - Palau de Congressos,POINT (-0.40182 39.49418),-0.401818,39.494184,None
7,Benicalap,emt,1331.0,Florista - Carrasqueta,POINT (-0.39621 39.49405),-0.396209,39.494049,None


In [9]:
# EMT 2

def coordenadas_long(x):
    if x is not None:
        return x['lon']
def coordenadas_lat(x):
    if x is not None:
        return x['lat']

# Añadir columnas
gdf_emt2['longitud'] = list(map(coordenadas_long, gdf_emt2.geo_point_2d))
gdf_emt2['latitud'] = list(map(coordenadas_lat, gdf_emt2.geo_point_2d))

# Quitar columnas
columnas_eliminar = ['codvia', 'numportal', 'suprimida', 'proximas_llegadas', 'geo_point_2d']
gdf_emt2.drop(columnas_eliminar, axis=1, inplace=True)

# Renombrar columnas
gdf_emt2.rename(columns={'id_parada': 'id'}, inplace=True)
gdf_emt2.rename(columns={'denominacion': 'nombre'}, inplace=True)
gdf_emt2.rename(columns={'lineas': 'linea'}, inplace=True)


gdf_emt2.head()


,id,nombre,linea,geometry,longitud,latitud
0,2180.0,Gavines (parell) - Platja (2180),25,POINT (-0.28923 39.29760),-0.289232,39.297596
1,2240.0,Actor Antonio Ferrandis (imparell) - En Corts ...,99,POINT (-0.36244 39.44855),-0.362439,39.448552
2,2032.0,Les Corts (2032),"11,16,26,6,80,94,N10",POINT (-0.37427 39.47872),-0.374268,39.478722
3,2194.0,Gavines - Botavara (2194),25,POINT (-0.29455 39.30528),-0.294552,39.305285
4,2193.0,Gavines - Les dunes (2193),25,POINT (-0.29169 39.30210),-0.291694,39.302102


##### ¡menudo marrón!

In [10]:
# for i, row in gdf_emt.iterrows():
#     print(row.municipio,'>>', row.nombre)
    
# for i, row in enumerate(gdf_emt2['nombre']):
#     print(row, gdf_emt2['linea'][i])
    
# for x, y in zip(sorted(gdf_emt.intento1), sorted(gdf_emt2.intento1)):
#     print(x, "||", y)



with open(emt, 'rb') as f:
    resultado = chardet.detect(f.read())

# codificación detectada
print(resultado)


{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


Queremos concatenar los dos ficheros de EMT porque uno contiene las líneas mientras que otro contiene la ubicación. El problema está en el nombre de la parada, así como la longitud, porque no hemos encontrado el encoding que permita tomar los acentos en gdf_emt y la longitud de gdf_emt2 es menor que gdf_emt. 

En este apartado `¡menudo marrón!` hemos querido conservar las posibilidades que hemos considerado para concatenar

Nota: esto no funciona, lo he intentado

    >> pd.merge(gdf_emt, gdf_emt2, on=['longitud', 'latitud'], how='inner')
    >> pd.merge(gdf_emt, gdf_emt2, on="geometry", how='inner')

In [11]:
def mis_palabras(palabra):
    """
    función que elimina los paréntesis y el contenido interno de los mismos. también se podría quitar el guión pero
    al final hemos decidirlo conservarlo
    """
    a = re.sub(r"\([^()]+\)", "", palabra) # | -
#     b = a.split()
#     c = set(b)
    return a.strip()

def similitud(x, y):
    """
    función que devuelve true en el caso que encuentre dos cadenas similares al 0.94%
    """
    
    s = SequenceMatcher(None, x, y).ratio()
    if s>0.94:
        return True
    else:
        return False
    return(s)

def similitud_1(x, y):
    """
    función que devuelve true en el caso que encuentre dos cadenas similares al 100%. es equivalente al ==
    """
    
    s = SequenceMatcher(None, x, y).ratio()
    if s==1:
        return True
    else:
        return False

In [12]:
# Reformateamos los nombres de las paradas de emt para quitar signos innecesarios

gdf_emt2["intento1"] = list(map(mis_palabras, gdf_emt2['nombre'])) 
gdf_emt["intento1"] = list(map(mis_palabras, gdf_emt['nombre'])) 

In [13]:
# Agregamos una columna con los índices

gdf_emt2["indices"] = [i for i in range(len(gdf_emt2))]
gdf_emt["indices"] = [i for i in range(len(gdf_emt))]

In [14]:
# Primera aproximación: calcular la similitud de las cadenas y crear un diccionario que asocie las más comunes

# true_pals_indice = {}
# for i, x in zip(gdf_emt.indices, gdf_emt.intento1):
#     true_pals_indice[i] = []
#     for j, y in zip(gdf_emt2.indices, gdf_emt2.intento1):
#         if similitud(x, y):
#             true_pals_indice[i].append(j)
        

In [15]:
# Segunda aproximación: equivalente a la primera pero agregando solo un índice

# for i, x in zip(gdf_emt.indices, gdf_emt.intento1):
#     true_pals_indice[i] = None
#     for j, y in zip(gdf_emt2.indices, gdf_emt2.intento1):
#         if similitud(x, y):
#             true_pals_indice[i] = j
#             gdf_emt2.at[j, "indices"] = i
        

In [16]:
# La forma de proceder en caso de la primera y segunda aproximación es la siguiente:

# # Lista para almacenar los GeoDataFrames concatenados
# concatenados = []

# # Recorres el diccionario
# for indice_gdf_emt, indices_gdf_emt2 in true_pals_indice.items():
#     # Seleccionas las filas que corresponden a los índices de cada GeoDataFrame
#     gdf_emt_seleccionado = gdf_emt[gdf_emt['indices'] == indice_gdf_emt]
#     gdf_emt2_seleccionado = gdf_emt2[gdf_emt2['indices'].isin(indices_gdf_emt2)]
#     # Concatenas los GeoDataFrames seleccionados
#     gdf_concatenado = gpd.GeoDataFrame(pd.concat([gdf_emt_seleccionado, gdf_emt2_seleccionado], axis=0))
#     # Agregas el GeoDataFrame concatenado a la lista
#     concatenados.append(gdf_concatenado)

# # Concatenas los GeoDataFrames de la lista
# resultado = pd.concat(concatenados, axis=0)

# print(resultado)

¡Si quieres ejecutarlo todo, descomenta estas dos celdas de abajo!

In [17]:
# Tercera aproximación: comparamos directamente la columna de coordenadas. Por alguna razón le cuesta cargar

"""
for i, xrow in gdf_emt.iterrows():
    for j, yrow in gdf_emt2.iterrows():
        if str(xrow.geometry)== str(yrow.geometry):
            gdf_emt2.at[j, "indices"] = i
"""         


'\nfor i, xrow in gdf_emt.iterrows():\n    for j, yrow in gdf_emt2.iterrows():\n        if str(xrow.geometry)== str(yrow.geometry):\n            gdf_emt2.at[j, "indices"] = i\n'

In [18]:
# Concatenamos los índices resultantes de ser idénticos
"""
df_concatenado_por_geometry = pd.merge(gdf_emt, gdf_emt2, on="indices", how='inner')
df_concatenado_por_geometry.to_csv("Datos transporte/emt_concatenado_geometry.csv", index=False) # exportar a geojson no va
"""

'\ndf_concatenado_por_geometry = pd.merge(gdf_emt, gdf_emt2, on="indices", how=\'inner\')\ndf_concatenado_por_geometry.to_csv("Datos transporte/emt_concatenado_geometry.csv", index=False) # exportar a geojson no va\n'

----

### Arreglar datos faltantes

In [19]:
# Nos aseguramos que todos tienen los mismos elementos (tiene que haber 8)

smet = set(gdf_met.columns)
semt = set(gdf_emt.columns)
sren = set(gdf_ren.columns)

smet&semt&sren

{'geometry',
 'id',
 'latitud',
 'linea',
 'longitud',
 'municipio',
 'nombre',
 'tipo'}

In [20]:
# gdf_met.to_file('metrovalencia.geojson', driver='GeoJSON')  
# gdf_emt.to_file('emt.geojson', driver='GeoJSON')  
# gdf_ren.to_file('renfe.geojson', driver='GeoJSON')  

In [21]:
# METROVALENCIA

# Falta municipio
gdf_met.head()

# for i, m in enumerate(gdf_met['nombre']):
#     print(i+1, m)
    
municipios_met = {}

def anyadir_municipio(estacion, municipio, dic=municipios_met, estaciones = set(gdf_met['nombre'])):
    if estacion in dic:
        print('YA EXISTE', estacion)
    elif estacion not in estaciones :
        print('NO EXISTE', estacion)
    else:
        dic[estacion] = municipio

In [22]:
## https://www.metrovalencia.es/es/consulta-estaciones/

anyadir_municipio('Mas del Rosari', 'La Coma - València')
anyadir_municipio('La Cova', 'Manises')
anyadir_municipio('Aeroport', 'Manises')
anyadir_municipio('Almàssera', 'Almàssera')
anyadir_municipio('Colón', 'Sant Francesc - València')
anyadir_municipio('Ayora', 'Ayora - València')
anyadir_municipio('Bétera', 'Bétera')
anyadir_municipio('Torrent Avinguda', 'Torrent')
anyadir_municipio('Alberic', 'Alberic')
anyadir_municipio('Carlet', 'Carlet')
anyadir_municipio('Col·legi El Vedat', 'Torrent')
anyadir_municipio('Moreres', 'Moreres - València')
anyadir_municipio('Sant Isidre', 'Sant Isidre - València')
anyadir_municipio('Sant Miquel dels Reis', 'Orriols - València')
anyadir_municipio('La Presa', 'Manises')
anyadir_municipio('Masia de Traver', 'Riba-roja de Túria')
anyadir_municipio('Jesús', 'La Raiosa - València')
anyadir_municipio('Patraix', 'Patraix - València')
anyadir_municipio('Omet', 'Picassent')
anyadir_municipio('Paiporta', 'Paiporta')
anyadir_municipio('Burjassot - Godella', 'Burjassot')
anyadir_municipio('Godella', 'Godella')
anyadir_municipio('Horta Vella', 'Bétera')
anyadir_municipio('Alfauir', 'Sant Llorenç - València')
anyadir_municipio('Parc Científic', 'Paterna')
anyadir_municipio('Tomás y Valiente', 'Paterna')
anyadir_municipio('Francesc Cubells', 'Grao - València')
anyadir_municipio('Grau - La Marina', 'Grao - València')
anyadir_municipio('Tossal del Rei', 'Torrefiel - València')
anyadir_municipio('Av. del Cid', 'València')
anyadir_municipio('Llíria', 'Llíria')
anyadir_municipio('Àngel Guimerà', 'Arrancapins - València')
anyadir_municipio('Campament', 'Paterna')
anyadir_municipio('Picanya', 'Picanya')
anyadir_municipio('Albalat dels Sorells', 'Albalat dels Sorells')
anyadir_municipio('Reus', 'Marxalenes - València')
anyadir_municipio('Trànsits', 'Benicalap - València')
anyadir_municipio('Benicalap', 'Benicalap - València')
anyadir_municipio('La Carrasca', 'La Carrasca - València')
anyadir_municipio('Dr. Lluch', 'Cabanyal-Canyamelar - València')
anyadir_municipio('Universitat Politècnica', 'La Carrasca - València')
anyadir_municipio('Alboraia Peris Aragó', 'Alboraia')
anyadir_municipio('Fondo de Benaguasil', 'Benaguasil')
anyadir_municipio('Gallipont - Torre del Virrei', 'L\'Eliana')
anyadir_municipio('Fuente del Jarro', 'Paterna')
anyadir_municipio('L\'Alcúdia', 'L\'Alcúdia')
anyadir_municipio('Sant Ramon', 'Picassent')
anyadir_municipio('Garbí', 'Benicalap - València')
anyadir_municipio('Florista', 'Benicalap - València')
anyadir_municipio('Neptú', 'Grao - València')
anyadir_municipio('Machado', 'Benimaclet - València')
anyadir_municipio('Roses', 'Manises')
anyadir_municipio('Amistat', 'Amistat - València')
anyadir_municipio('Aragó', 'Mestalla - València')
anyadir_municipio('Castelló', 'Castelló')
anyadir_municipio('Alginet', 'Alginet')
anyadir_municipio('Quatre Carreres', 'La Fonteta de Sant Lluis - València')
anyadir_municipio('Oceanogràfic', 'Ciutat de les Arts i les Ciències - València')
anyadir_municipio('Palau de Congressos', 'Benicalap - València')
anyadir_municipio('Campus', 'Burjassot')
anyadir_municipio('Entrepins', 'L\'Eliana')
anyadir_municipio('Burjassot', 'Burjassot')
anyadir_municipio('Mislata', 'Mislata')
anyadir_municipio('Túria', 'Campanar - València')
anyadir_municipio('La Vallesa', 'Paterna')
anyadir_municipio('Montortal', 'L\'Alcúdia')
anyadir_municipio('Rocafort', 'Rocafort')
anyadir_municipio('Moncada - Alfara', 'Moncada - Alfara')
anyadir_municipio('Masies', 'Moncada - Alfara')
anyadir_municipio('Russafa', 'Russafa - València')
anyadir_municipio('Orriols', 'Orriols - València')
anyadir_municipio('Pont de Fusta', 'Morvedre - València')
anyadir_municipio('Vicent Andrés Estellés', 'Burjassot')
anyadir_municipio('À Punt', 'Burjassot')
anyadir_municipio('La Coma', 'Paterna')
anyadir_municipio('Estadi Ciutat de València', 'Orriols - València')
anyadir_municipio('Xàtiva', 'Sant Francesc - València')
anyadir_municipio('Quart de Poblet', 'Quart de Poblet')
anyadir_municipio('Meliana', 'Meliana')
anyadir_municipio('Foios', 'Foios')
anyadir_municipio('Les Carolines - Fira', 'Benimàmet - València')
anyadir_municipio('Canyamelar', 'Cabanyal-Canyamelar - València')
anyadir_municipio('Font Almaguer', 'Benifaio')
anyadir_municipio('Sagunt', 'València')
anyadir_municipio('Campanar', 'Campanar - València')
anyadir_municipio('Massamagrell', 'Massamagrell')
anyadir_municipio('Massarrojos', 'Massarrojos')
anyadir_municipio('La Pobla de Farnals', 'La Pobla de Farnals')
anyadir_municipio('Facultats - Manuel Broseta', 'Exposició - València')
anyadir_municipio('El Clot', 'L\'Eliana')
anyadir_municipio('Safranar', 'Patraix - València')
anyadir_municipio('Ll. Llarga - Terramelar', 'Paterna')
anyadir_municipio('Salt de l\'Aigua', 'Manises')
anyadir_municipio('Bailén', 'La Roqueta - València')
anyadir_municipio('Rafelbunyol', 'Rafelbunyol')
anyadir_municipio('Alacant', 'Russafa - València')
anyadir_municipio('Faitanar', 'Quart de Poblet')
anyadir_municipio('L\'Eliana', 'L\'Eliana')
anyadir_municipio('Riba-roja de Túria', 'Riba-roja de Túria')
anyadir_municipio('Montesol', 'L\'Eliana')
anyadir_municipio('Picassent', 'Picassent')
anyadir_municipio('Torrent', 'Torrent')
anyadir_municipio('Ciutat Arts i Ciències - Justícia', 'Ciutat Arts i Ciències  - València')
anyadir_municipio('Natzaret', 'Natzaret - València')
anyadir_municipio('La Granja', 'Burjassot')
anyadir_municipio('Sant Joan', 'Burjassot')
anyadir_municipio('Museros', 'Museros')
anyadir_municipio('Alboraia Palmaret', 'Alboraia')
anyadir_municipio('València la Vella', 'Riba-roja de Túria')
anyadir_municipio('València Sud', 'Faitanar - València')
anyadir_municipio('Beniferri', 'Benicalap - València')
anyadir_municipio('Pl. Espanya', 'La Roqueta - València')
anyadir_municipio('Benimodo', 'Benimodo')
anyadir_municipio('Ausiàs March', 'Carlet')
anyadir_municipio('Espioca', 'Picassent')
anyadir_municipio('Seminari - CEU', 'Moncada')
anyadir_municipio('Amado Granell - Montolivet', 'Montolivet - València')
anyadir_municipio('Tarongers - Ernest Lluch', 'La Carrasca - València')
anyadir_municipio('Mislata Almassil', 'Mislata')
anyadir_municipio('Cabanyal', 'Cabanyal - València')
anyadir_municipio('Nou d\'Octubre', 'Soternes - València')
anyadir_municipio('Alameda', 'La Xerea - València')
anyadir_municipio('Manises', 'Manises')
anyadir_municipio('Paterna', 'Paterna')
anyadir_municipio('Marítim', 'Ayora - València')
anyadir_municipio('Benimàmet', 'Benimàmet - València')
anyadir_municipio('Cantereria', 'Burjassot')
anyadir_municipio('Benaguasil', 'Benaguasil')
anyadir_municipio('Realón', 'Picassent')
anyadir_municipio('Beteró', 'La Carrasca - València')
anyadir_municipio('Marxalenes', 'Marxalenes - València')
anyadir_municipio('Vicente Zaragozá', 'Ciutat Universitària - València')
anyadir_municipio('Trinitat', 'Benimaclet - València')
anyadir_municipio('La Pobla de Vallbona', 'La Pobla de Vallbona')
anyadir_municipio('Benimaclet', 'Benimaclet - València')
anyadir_municipio('La Canyada', 'Paterna')
anyadir_municipio('Santa Rita', 'Paterna')
anyadir_municipio('Massalavés', 'Massalavés')
anyadir_municipio('Empalme', 'Burjassot')
anyadir_municipio('La Cadena', 'Cabanyal-Canyamelar - València')
anyadir_municipio('Platja Malva-rosa', 'Cabanyal-Canyamelar - València')
anyadir_municipio('Platja les Arenes', 'Cabanyal-Canyamelar - València')

In [23]:
# Añadimos los cambios y fusionamos

df = pd.DataFrame([[key, municipios_met[key]] for key in municipios_met.keys()], columns=['nombre', 'municipio'])
gdf_met = gdf_met.merge(df, on="nombre")
del gdf_met['municipio_x']
gdf_met.rename(columns={'municipio_y': 'municipio'}, inplace=True)
gdf_met.head()

,id,nombre,linea,geometry,longitud,latitud,tipo,municipio
0,237118,Mas del Rosari,4,POINT (-0.43582 39.52496),-0.435825,39.524960,metrovalencia,La Coma - València
1,237200,La Cova,9,POINT (-0.48444 39.49889),-0.484444,39.498890,metrovalencia,Manises
2,237215,Aeroport,"3,5",POINT (-0.47492 39.49237),-0.474919,39.492367,metrovalencia,Manises
3,237216,Almàssera,3,POINT (-0.35427 39.51226),-0.354267,39.512257,metrovalencia,Almàssera
4,237142,Colón,"3,5,7,9",POINT (-0.37093 39.47015),-0.370928,39.470146,metrovalencia,Sant Francesc - València


In [24]:
# EMT
gdf_emt.head()

# Falta línea
# for i, n in enumerate(gdf_emt['nombre']):
#     print(i, n)
    
    
# Falta arreglar municipio
# for i, n in enumerate(gdf_emt['municipio']):
#     print(i, n)

# Véase ¡menudo marrón!  --> obtenemos el siguiente fichero
gdf_emt3 = pd.read_csv(emt3)
gdf_emt3.head()

,municipio,tipo,id_x,nombre_x,geometry_x,longitud_x,latitud_x,linea_x,intento1_x,indices,id_y,nombre_y,linea_y,geometry_y,longitud_y,latitud_y,intento1_y
0,Benicalap,emt,1067.0,Equador - Pintor Matarana,POINT (-0.392778225524496 39.49607883121975),-0.392778,39.496079,NaN,Equador - Pintor Matarana,1,1067.0,Equador - Pintor Matarana (1067),12,POINT (-0.3927782320996148 39.49607896649905),-0.392778,39.496079,Equador - Pintor Matarana
1,Benicalap,emt,1183.0,General Avil�s (par) - Dr. Nicasi Benlloch,POINT (-0.391460962857036 39.48803711704195),-0.391461,39.488037,NaN,General Avil�s - Dr. Nicasi Benlloch,2,2032.0,Les Corts (2032),"11,16,26,6,80,94,N10",POINT (-0.3742677624589383 39.47872157854415),-0.374268,39.478722,Les Corts
2,Benicalap,emt,1183.0,General Avil�s (par) - Dr. Nicasi Benlloch,POINT (-0.391460962857036 39.48803711704195),-0.391461,39.488037,NaN,General Avil�s - Dr. Nicasi Benlloch,2,1183.0,General Avilés - Dr. Nicasi Benlloch (1183),"64,94",POINT (-0.3914609581496043 39.488037243059374),-0.391461,39.488037,General Avilés - Dr. Nicasi Benlloch
3,Benicalap,emt,1319.0,Corts Valencianes - Palau de Congressos,POINT (-0.401818139798389 39.494183715692415),-0.401818,39.494184,NaN,Corts Valencianes - Palau de Congressos,3,1319.0,Corts Valencianes - Palau de Congressos (1319),"62,63,99,N3",POINT (-0.401818111537627 39.494183850192464),-0.401818,39.494184,Corts Valencianes - Palau de Congressos
4,Benicalap,emt,1331.0,Florista - Carrasqueta,POINT (-0.396208839095057 39.494049233440656),-0.396209,39.494049,NaN,Florista - Carrasqueta,4,1331.0,Florista - Carrasqueta (1331),28,POINT (-0.3962088210973265 39.49404940420439),-0.396209,39.494049,Florista - Carrasqueta


In [25]:
# Quitar columnas
columnas_eliminar = ['id_x', 'nombre_x', 'geometry_x', 'longitud_x', 'latitud_x', 'linea_x', 'intento1_x', 'intento1_y', 'indices']
gdf_emt3.drop(columnas_eliminar, axis=1, inplace=True)

# Renombrar columnas
gdf_emt3.rename(columns={'id_y': 'id'}, inplace=True)
gdf_emt3.rename(columns={'nombre_y': 'nombre'}, inplace=True)
gdf_emt3.rename(columns={'linea_y': 'linea'}, inplace=True)
gdf_emt3.rename(columns={'longitud_y': 'longitud'}, inplace=True)
gdf_emt3.rename(columns={'latitud_y': 'latitud'}, inplace=True)
gdf_emt3.rename(columns={'geometry_y': 'geometry'}, inplace=True)

# Modificar columna
gdf_emt3['geometry'] = gdf_emt3['geometry'].apply(loads) # Convierte las cadenas de texto en objetos geométricos
gdf_emt3 = gpd.GeoDataFrame(gdf_emt3, geometry='geometry') # Crea un GeoDataFrame a partir del DataFrame df
gdf_emt3['nombre'] = list(map(mis_palabras, gdf_emt3.nombre))

gdf_emt3.head()

,municipio,tipo,id,nombre,linea,geometry,longitud,latitud
0,Benicalap,emt,1067.0,Equador - Pintor Matarana,12,POINT (-0.39278 39.49608),-0.392778,39.496079
1,Benicalap,emt,2032.0,Les Corts,"11,16,26,6,80,94,N10",POINT (-0.37427 39.47872),-0.374268,39.478722
2,Benicalap,emt,1183.0,General Avilés - Dr. Nicasi Benlloch,"64,94",POINT (-0.39146 39.48804),-0.391461,39.488037
3,Benicalap,emt,1319.0,Corts Valencianes - Palau de Congressos,"62,63,99,N3",POINT (-0.40182 39.49418),-0.401818,39.494184
4,Benicalap,emt,1331.0,Florista - Carrasqueta,28,POINT (-0.39621 39.49405),-0.396209,39.494049


In [26]:
# RENFE
gdf_ren.head()
   
# Faltan estaciones? Viendo el mapa, no: quitamos los None
tiene_none = []
for i, n in enumerate(gdf_ren['nombre']):
    if n is None:
        tiene_none.append(i)
gdf_ren.drop(tiene_none, axis=0, inplace=True)


# Falta linea


# for i, n in enumerate(gdf_ren['municipio']):
#     print(i, n, n is None)

# gdf['nombre'] = gdf['municipio'].where(gdf['nombre'].isna(), gdf['nombre'])

c1 = ['Cullera', 'Valencia-Estacio del Nord', 'Xeraco', 'Alfafar-Benetusser', 'Catarroja', 'El Romani', 'Gandia', 'Platja i Grau de Gandia', 'Silla', 'Sueca', 'Tavernes de la Valldigna', 'Massanassa', 'Sollana']
c2 = ['Moixent', 'Vallada', 'Montesa', "L'Alcudia", 'Xativa', "L'Enova-Manuel", 'La Pobla Llarga', 'Carcaixent', 'Alzira', 'Algemesi', 'Benifaio', 'Valencia-Estacio del Nord', 'Alfafar-Benetusser', 'Catarroja', 'Silla', 'Massanassa']
c3 = ['Valencia Sant Isidre', 'Xirivella-Alqueries', 'Aldaia', 'Loriguilla-Reva', 'Circuit Ricardo Tormo', 'Cheste', 'Chiva', 'Buñol', 'Venta Mina-Siete Aguas', 'Siete Aguas', 'El Rebollar', 'Requena', 'San Antonio de Requena', 'Utiel']
c4 = ["Xirivella-L'Alter", 'Valencia Sant Isidre']
c5 = ['Valencia-La Font de Sant Lluis', 'Valencia-Cabanyal', 'Roca-Cuper', 'Albuixech', 'Massalfassar', 'El puig', 'Puçol', 'Sagunt', 'Les Valls', 'Almenara', 'La Llosa', 'Chilches', 'Moncofar', 'Nules-villavieja', 'Burriana-Alquerias del Niño Perdido', 'Vila-Real', 'Almassora', 'Castello de la Plana']
c6 = ['Valencia-La Font de Sant Lluis', 'Valencia-Cabanyal', 'Roca-Cuper', 'Albuixech', 'Massalfassar', 'El puig', 'Puçol', 'Sagunt', 'Algimia-Ciudad', 'Caudiel', 'Estivella-Albalat dels Tarongers', 'Gilet', 'Jerica-Viver', 'Navajas', 'Segorbe-Arrabal', 'Segorbe-Ciudad', 'Soneja']

for i, row in gdf_ren.iterrows():
    lineas = []
    if row.nombre in c1:
        lineas.append('C1')
    if row.nombre in c2:
        lineas.append('C2')
    if row.nombre in c3:
        lineas.append('C3')
    if row.nombre in c4:
        lineas.append('C4')
    if row.nombre in c5:
        lineas.append('C5')
    if row.nombre in c6:
        lineas.append('C6')
    lineas = ','.join(lineas)
    gdf_ren.at[i, 'linea'] = lineas

    
gdf_ren.head()

,id,municipio,latitud,longitud,geometry,tipo,linea,nombre
0,65005,Albuixech,39.5441373,-0.320388,POINT (-0.32039 39.54414),renfe,"C5,C6",Albuixech
1,65209,Almassora,39.9543523,-0.0775039,POINT (-0.07750 39.95435),renfe,C5,Almassora
2,65202,Almenara,39.7522591,-0.2156704,POINT (-0.21567 39.75226),renfe,C5,Almenara
3,66206,Buñol,39.4273336,-0.7875467,POINT (-0.78755 39.42733),renfe,C3,Buñol
4,64103,Carcaixent,39.1182487,-0.4547076,POINT (-0.45471 39.11825),renfe,C2,Carcaixent


In [27]:
# Comprobación
# for i, n in enumerate(gdf_ren['municipio']):
#     print(i, n, gdf_ren["linea"][i])


### Fusión

In [28]:
import geopandas as gpd

# Restablecer los índices de los GeoDataFrames para evitar índices duplicados
gdf_met.reset_index(drop=True, inplace=True)
gdf_emt3.reset_index(drop=True, inplace=True)
gdf_ren.reset_index(drop=True, inplace=True)

In [29]:
gdf = pd.concat([gdf_met,gdf_emt3, gdf_ren])
gdf.head()

C:\Users\34686\Downloads\Anaconda\lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


,id,nombre,linea,geometry,longitud,latitud,tipo,municipio
0,237118,Mas del Rosari,4,POINT (-0.43582 39.52496),-0.435825,39.52496,metrovalencia,La Coma - València
1,237200,La Cova,9,POINT (-0.48444 39.49889),-0.484444,39.49889,metrovalencia,Manises
2,237215,Aeroport,"3,5",POINT (-0.47492 39.49237),-0.474919,39.492367,metrovalencia,Manises
3,237216,Almàssera,3,POINT (-0.35427 39.51226),-0.354267,39.512257,metrovalencia,Almàssera
4,237142,Colón,"3,5,7,9",POINT (-0.37093 39.47015),-0.370928,39.470146,metrovalencia,Sant Francesc - València


In [30]:
gdf.to_file('Datos transporte/transporte.geojson', driver='GeoJSON')  

In [31]:
gdf

,id,nombre,linea,geometry,longitud,latitud,tipo,municipio
0,237118,Mas del Rosari,4,POINT (-0.43582 39.52496),-0.435825,39.52496,metrovalencia,La Coma - València
1,237200,La Cova,9,POINT (-0.48444 39.49889),-0.484444,39.49889,metrovalencia,Manises
2,237215,Aeroport,"3,5",POINT (-0.47492 39.49237),-0.474919,39.492367,metrovalencia,Manises
3,237216,Almàssera,3,POINT (-0.35427 39.51226),-0.354267,39.512257,metrovalencia,Almàssera
4,237142,Colón,"3,5,7,9",POINT (-0.37093 39.47015),-0.370928,39.470146,metrovalencia,Sant Francesc - València
...,...,...,...,...,...,...,...,...
61,67217,Soneja,C6,POINT (-0.43067 39.81202),-0.4306676,39.8120247,renfe,Soneja
62,64004,Vallada,C2,POINT (-0.68995 38.90899),-0.689953,38.908988,renfe,Vallada
63,66205,Venta Mina-Siete Aguas,C3,POINT (-0.87066 39.45407),-0.870661,39.454073,renfe,Venta Mina-Siete Aguas
64,64100,Xativa,C2,POINT (-0.52537 38.99183),-0.5253742,38.9918333,renfe,Xativa


In [32]:
for i, row in gdf.iterrows():
    print(f'{i} {row.nombre} << {row.municipio}')

0 Mas del Rosari << La Coma - València
1 La Cova << Manises
2 Aeroport << Manises
3 Almàssera << Almàssera
4 Colón << Sant Francesc - València
5 Ayora << Ayora - València
6 Bétera << Bétera
7 Torrent Avinguda << Torrent
8 Alberic << Alberic
9 Carlet << Carlet
10 Col·legi El Vedat << Torrent
11 Moreres << Moreres - València
12 Sant Isidre << Sant Isidre - València
13 Sant Miquel dels Reis << Orriols - València
14 La Presa << Manises
15 Masia de Traver << Riba-roja de Túria
16 Jesús << La Raiosa - València
17 Patraix << Patraix - València
18 Omet << Picassent
19 Paiporta << Paiporta
20 Burjassot - Godella << Burjassot
21 Godella << Godella
22 Horta Vella << Bétera
23 Alfauir << Sant Llorenç - València
24 Parc Científic << Paterna
25 Tomás y Valiente << Paterna
26 Francesc Cubells << Grao - València
27 Grau - La Marina << Grao - València
28 Tossal del Rei << Torrefiel - València
29 Av. del Cid << València
30 Llíria << Llíria
31 Àngel Guimerà << Arrancapins - València
32 Campament << Pater

1059 Tírig - Guàrdia Civil << El Saler
1060 Escultor Salzillo - Lluís Fenollet << La Fontsanta
1061 Plaça d'Espanya << La Fontsanta
1062 Valladolid - Dolores Marqués << Sant Llorens
1063 Duc de Mandas - Alfahuir << Sant Llorens
1064 Hernàn Cortès - Ciril Amorós << Els Orriols
1065 Primat Reig - Motilla del Palancar << Els Orriols
1066 Primat Reig - Constitució << Els Orriols
1067 Duc de Mandas - Constitució << Els Orriols
1068 Sant Francesc de Borja - Jesús << Els Orriols
1069 Primat Reig - Sant Vicent de Paül << Els Orriols
1070 Pare Viñas - Duc de Mandas << Els Orriols
1071 Sant Vicent Màrtir - Fernández de Mesa << Els Orriols
1072 Sant Joan Bosco - Daroca << Els Orriols
1073 Pérez Galdós - Mestre Guerrero << Els Orriols
1074 Gavines - Entrada Ventura << El Perellonet
1075 Gavines - Entrada a la Baixada del Pantà << El Perellonet
1076 Camí de Montcada - Entrada Camí Carpesa << El Perellonet
1077 Gavines - Dofins << El Perellonet
1078 Tres Creus - Dr. Pérez Feliu << Vara de Quart
1079

**Nota**: existen algunas diferencias entre cómo llama EMT a los barrios y cómo los he nombrado yo, por ejemplo:
* La Fonteta de Sant Lluís <=>  Fontsana
* Ciutat de les Arts i les Ciències  <=>   La Ciutat de Les Arts i de Les Ciencies
* Orriols  <=>  Els Orriols

La unificación de nomenclatura en barrios se lo dejo a quien quiera seguir.